# Lesson 1: Advanced RAG Pipeline

Basic RAG pipeline consists of three phases

* Ingestion (Documents to chunks to embeddings to index)
* Retrieval (Query to Index to top K)
* Synthesis (Top K, Query to LLM to Response)


In [1]:
import utils

import os
import openai
openai.api_key = utils.get_openai_api_key()

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


In [2]:
from llama_index import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["./eBook-How-to-Build-a-Career-in-AI.pdf"]   # you can use your own PDF
).load_data()

In [3]:
print(type(documents), "\n")
print(len(documents), "\n")
print(type(documents[0]))
print(documents[0])

<class 'list'> 

41 

<class 'llama_index.schema.Document'>
Doc ID: 6dfaa855-8fc6-48a9-8c01-d9339dc5381d
Text: PAGE 1Founder, DeepLearning.AICollected Insights from Andrew Ng
How to  Build Your Career in AIA Simple Guide


## Basic RAG pipeline

In [4]:
from llama_index import Document

document = Document(text="\n\n".join([doc.text for doc in documents]))

In [5]:
from llama_index import VectorStoreIndex
from llama_index import ServiceContext
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)
service_context = ServiceContext.from_defaults(
    llm=llm, embed_model="local:BAAI/bge-small-en-v1.5"  # Huggingface BGE Small
)
index = VectorStoreIndex.from_documents([document],
                                        service_context=service_context)

In [6]:
query_engine = index.as_query_engine()

In [7]:
response = query_engine.query(
    "What are steps to take when finding projects to build your experience?"
)
print(str(response))

Develop a side hustle, ensure the project will help you grow technically, collaborate with good teammates, and consider if the project can be a stepping stone to larger projects.


## Evaluation setup using TruLens

Query to Context to Response to Query

Query to Context : Context Relevance: Is the retrieved context relevant to the query.
Context to Response : Groundedness: Is the response supported by context?
Response to Query: Is the response relevant to the query?


In [8]:
eval_questions = []
with open('eval_questions.txt', 'r') as file:
    for line in file:
        # Remove newline character and convert to integer
        item = line.strip()
        print(item)
        eval_questions.append(item)

What are the keys to building a career in AI?
How can teamwork contribute to success in AI?
What is the importance of networking in AI?
What are some good habits to develop for a successful career?
How can altruism be beneficial in building a career?
What is imposter syndrome and how does it relate to AI?
Who are some accomplished individuals who have experienced imposter syndrome?
What is the first step to becoming good at AI?
What are some common challenges in AI?
Is it normal to find parts of AI challenging?


In [9]:
# You can try your own question:
new_question = "What is the right AI job for me?"
eval_questions.append(new_question)

In [10]:
print(eval_questions)

['What are the keys to building a career in AI?', 'How can teamwork contribute to success in AI?', 'What is the importance of networking in AI?', 'What are some good habits to develop for a successful career?', 'How can altruism be beneficial in building a career?', 'What is imposter syndrome and how does it relate to AI?', 'Who are some accomplished individuals who have experienced imposter syndrome?', 'What is the first step to becoming good at AI?', 'What are some common challenges in AI?', 'Is it normal to find parts of AI challenging?', 'What is the right AI job for me?']


In [11]:
from trulens_eval import Tru
tru = Tru()

tru.reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


For the classroom, we've written some of the code in helper functions inside a utils.py file.  
- You can view the utils.py file in the file directory by clicking on the "Jupyter" logo at the top of the notebook.
- In later lessons, you'll get to work directly with the code that's currently wrapped inside these helper functions, to give you more options to customize your RAG pipeline.

In [12]:
from utils import get_prebuilt_trulens_recorder

tru_recorder = get_prebuilt_trulens_recorder(query_engine,
                                             app_id="Direct Query Engine")

In [13]:
with tru_recorder as recording:
    for question in eval_questions:
        response = query_engine.query(question)

In [14]:
records, feedback = tru.get_records_and_feedback(app_ids=[])

In [15]:
records.head()

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,Answer Relevance,Context Relevance,Answer Relevance_calls,Context Relevance_calls,Groundedness,Groundedness_calls,latency,total_tokens,total_cost
0,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_cbf1b6cf1860ab5ef67caa4d24e59ec6,"""What are the keys to building a career in AI?""","""Learning foundational technical skills, worki...",-,"{""record_id"": ""record_hash_cbf1b6cf1860ab5ef67...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-10-17T15:49:48.626756"", ""...",2024-10-17T15:49:50.505532,1.0,1.00,[{'args': {'prompt': 'What are the keys to bui...,[{'args': {'prompt': 'What are the keys to bui...,NaN,NaN,1,2078,0.003148
1,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_ac402d3c67e765104a9dd8ae9964184d,"""How can teamwork contribute to success in AI?""","""Teamwork can contribute to success in AI by a...",-,"{""record_id"": ""record_hash_ac402d3c67e765104a9...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-10-17T15:49:50.716634"", ""...",2024-10-17T15:49:52.761846,1.0,0.50,[{'args': {'prompt': 'How can teamwork contrib...,[{'args': {'prompt': 'How can teamwork contrib...,0.933333,[{'args': {'source': 'Hopefully the previous c...,2,1697,0.002582
2,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_c0929e471dc678d14ba411f0d0fa658f,"""What is the importance of networking in AI?""","""Networking is crucial in AI as it helps indiv...",-,"{""record_id"": ""record_hash_c0929e471dc678d14ba...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-10-17T15:49:52.962644"", ""...",2024-10-17T15:49:54.897667,1.0,0.45,[{'args': {'prompt': 'What is the importance o...,[{'args': {'prompt': 'What is the importance o...,NaN,NaN,1,1693,0.002573
3,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_60409eabf292a26cc7aa6c47dffee5c2,"""What are some good habits to develop for a su...","""Developing good habits in areas such as eatin...",-,"{""record_id"": ""record_hash_60409eabf292a26cc7a...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-10-17T15:49:55.101150"", ""...",2024-10-17T15:49:56.557067,1.0,0.95,[{'args': {'prompt': 'What are some good habit...,[{'args': {'prompt': 'What are some good habit...,NaN,NaN,1,1631,0.002465
4,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_89e69f672d915f0efdd18a06a443d6dd,"""How can altruism be beneficial in building a ...","""Aiming to lift others during every step of on...",-,"{""record_id"": ""record_hash_89e69f672d915f0efdd...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-10-17T15:49:56.760130"", ""...",2024-10-17T15:49:58.065107,1.0,0.55,[{'args': {'prompt': 'How can altruism be bene...,[{'args': {'prompt': 'How can altruism be bene...,NaN,NaN,1,1616,0.002435


In [16]:
# launches on http://localhost:8501/
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at https://s172-29-80-181p38560.lab-aws-production.deeplearning.ai/ .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

## Advanced RAG pipeline

### 1. Sentence Window retrieval


Embedding and retrieving, sentences are placed in the larger window of sentences  around the original retrieved sentence.  Provides more context to the LLMs.

In [17]:
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)

In [18]:
from utils import build_sentence_window_index

sentence_index = build_sentence_window_index(
    document,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="sentence_index"
)

In [19]:
from utils import get_sentence_window_query_engine

sentence_window_engine = get_sentence_window_query_engine(sentence_index)

In [20]:
window_response = sentence_window_engine.query(
    "how do I get started on a personal project in AI?"
)
print(str(window_response))

To get started on a personal project in AI, you can begin by identifying a project that aligns with your career goals and interests. It's important to choose a project that is responsible, ethical, and beneficial to people. Once you have selected a project, you can follow the steps outlined in the chapters provided, such as scoping the project, executing it with an eye toward career development, and building a portfolio that shows skill progression. By following these guidelines, you can embark on a personal AI project that not only enhances your skills but also makes a positive impact in the field.


In [21]:
tru.reset_database()

tru_recorder_sentence_window = get_prebuilt_trulens_recorder(
    sentence_window_engine,
    app_id = "Sentence Window Query Engine"
)

In [22]:
for question in eval_questions:
    with tru_recorder_sentence_window as recording:
        response = sentence_window_engine.query(question)
        print(question)
        print(str(response))

What are the keys to building a career in AI?
The keys to building a career in AI involve learning foundational technical skills, working on projects, finding a job, and being part of a supportive community.
How can teamwork contribute to success in AI?
Teammates play a crucial role in the success of AI projects. Working collaboratively with colleagues who are dedicated, continuously learning, and focused on building AI for the benefit of all can positively influence individual performance. The ability to work effectively in a team, leveraging each member's strengths and insights, can lead to improved project outcomes and overall success in the field of AI.
What is the importance of networking in AI?
Networking in AI is crucial as it can provide valuable insights, guidance, and opportunities for individuals looking to advance in the field. By connecting with professionals who have experience in AI, individuals can gain knowledge about the industry, potential career paths, and current t

In [23]:
tru.get_leaderboard(app_ids=[])

,Answer Relevance,latency,total_cost
app_id,,,
Sentence Window Query Engine,1.0,1.909091,0.000812


In [24]:
# launches on http://localhost:8501/
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path: https://s172-29-80-181p38560.lab-aws-production.deeplearning.ai/


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

### 2. Auto-merging retrieval

Hierarchy of parent and child nodes of a certin chunk size. Retrieves nodes to the larger parent nodes. 


In [25]:
from utils import build_automerging_index

automerging_index = build_automerging_index(
    documents,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="merging_index"
)

In [26]:
from utils import get_automerging_query_engine

automerging_query_engine = get_automerging_query_engine(
    automerging_index,
)

In [27]:
auto_merging_response = automerging_query_engine.query(
    "How do I build a portfolio of AI projects?"
)
print(str(auto_merging_response))

> Merging 1 nodes into parent node.
> Parent node id: d5343e03-3838-42f9-aef9-045ad85239f8.
> Parent node text: PAGE 21Building a Portfolio of 
Projects that Shows 
Skill Progression CHAPTER 6
PROJECTS

> Merging 1 nodes into parent node.
> Parent node id: d4b57471-d74c-403c-93a6-629eb2da9a50.
> Parent node text: PAGE 21Building a Portfolio of 
Projects that Shows 
Skill Progression CHAPTER 6
PROJECTS

Building a portfolio of AI projects involves showcasing a progression from simple to complex undertakings over time. It is important to be able to communicate your thinking effectively to demonstrate the value of your work and gain trust from others. Identifying worthwhile ideas to work on is a crucial skill for an AI architect, and gaining experience across various industries by applying machine learning can help in building a diverse portfolio.


In [28]:
tru.reset_database()

tru_recorder_automerging = get_prebuilt_trulens_recorder(automerging_query_engine,
                                                         app_id="Automerging Query Engine")

In [29]:
for question in eval_questions:
    with tru_recorder_automerging as recording:
        response = automerging_query_engine.query(question)
        print(question)
        print(response)

A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x7fcf9188ba30 is calling an instrumented method <function BaseQueryEngine.query at 0x7fd107f0b760>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7fd0182cee00) using this function.
A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x7fcf9188ba30 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x7fd102da2b90>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7fd0182cee00) using this function.
A new object of type <class 'llama_index.retrievers.auto_merging_retriever.AutoMergingRetriever'> at 0x7fcffc11d450 is calling an instrumented method <function BaseRetriever.retrieve at 0x7fd107f0ab00>. The path of this call may be incorrect.
Guessing path of new object is app.retriever based on other object (0x7

> Merging 2 nodes into parent node.
> Parent node id: 3b795f4f-59c3-4413-b9fe-36cc71083d7a.
> Parent node text: PAGE 3Table of 
ContentsIntroduction: Coding AI is the New Literacy.
Chapter 1: Three Steps to Ca...

> Merging 1 nodes into parent node.
> Parent node id: 72841c8f-ab23-4c82-b564-df92b10e4e72.
> Parent node text: PAGE 3Table of 
ContentsIntroduction: Coding AI is the New Literacy.
Chapter 1: Three Steps to Ca...



A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x7fcf9188b9a0 is calling an instrumented method <function CompactAndRefine.get_response at 0x7fd107503250>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x7fd0182cedd0) using this function.
A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x7fcf9188b9a0 is calling an instrumented method <function Refine.get_response at 0x7fd106747010>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x7fd0182cedd0) using this function.
A new object of type <class 'llama_index.llm_predictor.base.LLMPredictor'> at 0x7fcfe02771c0 is calling an instrumented method <function LLMPredictor.predict at 0x7fd1143fd240>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthes

What are the keys to building a career in AI?
Learning foundational technical skills, working on projects, finding a job, and being part of a community are key steps to building a career in AI. Additionally, collaborating with others, influencing, and being influenced by others are critical aspects for success in AI.


A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x7fcf9188b9a0 is calling an instrumented method <function Refine.get_response at 0x7fd106747010>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x7fd0182cedd0) using this function.
A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x7fcf9188ba30 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x7fd102da2b90>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7fd0182cee00) using this function.


...

What are some common challenges in AI?
Some common challenges in AI include the highly iterative nature of AI projects, uncertainty in planning due to not knowing in advance how long it will take to achieve target accuracy, and facing technical challenges that may lead to feelings of imposter syndrome.
> Merging 3 nodes into parent node.
> Parent node id: 0a7c2698-2753-4051-a403-523fd10538db.
> Parent node text: PAGE 38Before we dive into the final chapter of this book, I’d like to address the serious matter...

> Merging 1 nodes into parent node.
> Parent node id: f1c58ee4-e106-484a-97f2-776743f7dca0.
> Parent node text: PAGE 38Before we dive into the final chapter of this book, I’d like to address the serious matter...



A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x7fcf9188b9a0 is calling an instrumented method <function Refine.get_response at 0x7fd106747010>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x7fd0182cedd0) using this function.
A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x7fcf9188ba30 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x7fd102da2b90>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7fd0182cee00) using this function.


Is it normal to find parts of AI challenging?
It is normal to find parts of AI challenging.
> Merging 1 nodes into parent node.
> Parent node id: 76d6e572-6e80-4853-a2f5-948aee081cc2.
> Parent node text: PAGE 31Finding the Right 
AI Job for YouCHAPTER 9
JOBS

> Merging 1 nodes into parent node.
> Parent node id: 7e97ac9c-07eb-407a-a19e-8d406a40d7e9.
> Parent node text: If you’re leaving 
a job, exit gracefully. Give your employer ample notice, give your full effort...

> Merging 1 nodes into parent node.
> Parent node id: 50bf2a26-4647-4802-8682-7fe354f355ea.
> Parent node text: PAGE 28Using Informational 
Interviews to Find 
the Right JobCHAPTER 8
JOBS

> Merging 1 nodes into parent node.
> Parent node id: 0b2a2947-49ee-4d5c-aded-f7e5c55e0984.
> Parent node text: PAGE 31Finding the Right 
AI Job for YouCHAPTER 9
JOBS

> Merging 1 nodes into parent node.
> Parent node id: 25ddaedd-002d-41fe-94b9-c3a05b32fd71.
> Parent node text: PAGE 28Using Informational 
Interviews to Find 
the Right

A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x7fcf9188b9a0 is calling an instrumented method <function Refine.get_response at 0x7fd106747010>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x7fd0182cedd0) using this function.


What is the right AI job for me?
The right AI job for you would likely involve transitioning into a role or industry that aligns with your current expertise or experience. If you are new to AI, it may be easier to start by switching either roles or industries first before attempting to switch both simultaneously.


In [30]:
tru.get_leaderboard(app_ids=[])

,Answer Relevance,latency,total_cost
app_id,,,
Automerging Query Engine,0.966667,3.0,0.000863


In [31]:
# launches on http://localhost:8501/
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path: https://s172-29-80-181p38560.lab-aws-production.deeplearning.ai/


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>